In [1]:
#read in cassis 
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator, List

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

from allennlp.predictors.predictor import Predictor
import spacy

PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"

BERT_PATH=os.path.join(  PATH, "tests/test_files/models", "bert_model", "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH, "tests/test_files/models", "spacy_model", "spacy-textcat" )

TEMPLATE_PATH=os.path.join( PATH, "tests/test_files/templates/out.html.template" )

sys.path.append( os.path.join( PATH ) )

with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/double_nested_list_response.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
#    response = json.load(json_file)  

#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32011D1208-input.json')) as json_file:
#    response = json.load(json_file)  
    
with open( os.path.join( PATH, 'tests/test_files/testing_bug/32020D0701-input.xmi_outputPar_det.json')) as json_file:
    response = json.load(json_file)  
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32020D0701-input.xmi_outputRO.json')) as json_file:
#    response = json.load(json_file)  
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/testing_bug2.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem )


In [2]:
from src.transform import ListTransformer
from src.reporting_obligations import ReportingObligationsFinder

from src.transform import get_other_lines, transform_lines, flatten_offsets

print( f"loading AllenNLP predictor from {BERT_PATH}" )
bert_model = Predictor.from_path( BERT_PATH )

print( f"loading spacy model from {SPACY_PATH}" )
nlp=spacy.load( SPACY_PATH )

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

transformer = ListTransformer( cas )
transformer.add_list_view( OldSofaID )
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )

loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat
ARGS[] ARG1 The full amount of the Union ’s macro‐financial assistance
ARGS[to] ARG2 to each partner
ARGM[] ARGM-MNR in the form of loans
Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/testing_bug2.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template


In [4]:
print( cas.get_view( 'ReportingObligationsView' ).sofa_string )

<!doctype html>
<style>

    p {
        margin: 0.75em 0px;
        line-height: 2em;
        white-space: nowrap;
    }

    span {
        border: 2px solid currentColor;
        border-radius: 5px;
        padding: 5px;
    }

    span {
        background: lightblue;
        color: royalblue;
    }

    .V {
        background: yellow;
        color: orangered;
        font-weight: bold;
    }

    .ARG0,.ARG1,.ARG2,.C-ARG1 {
        color: black;
        background: white;
    }

    .ARGM-TMP {
        color: green;
        background: lightgreen;
    }

    .ARGM-NEG {
        color: pink;
        background: darkred;
        border-color: darkred;
    }

    .PUNCT {
        color: silver;
        background: white;
        border: none; padding: 0;
    }

    .ARG2 {
        border-style: dotted;
    }

    .ARG1, .C-ARG1 {
        color: white;
        background: black;
        border-color: black;
        border-style: solid;
    }

    .ARG3, .ARGM-PRP, .ARGM-PRD, .ARGM-P

In [3]:
lines, offsets=get_other_lines( cas , OldSofaID, seek_vbtt, 'root', paragraph_type=paragraph_type )
flatten_offsets( offsets )

lines, offsets =postprocess_nested_lines( lines, offsets  )
assert( len( lines ) ) == len( offsets )
print( len( lines ) )

get other lines
get other lines
get other lines
get other lines
get other lines
656


In [7]:
import time
start=time.time()
    
transformed_lines, transformed_lines_offsets = transform_lines( lines, offsets )
end=time.time()
print( end-start )

0.09647393226623535


In [8]:
transformed_lines

['28.8.2008',
 'EN',
 'Official Journal of the European Union',
 'L 229/5',
 'COMMISSION DECISION',
 'of 13 August 2008',
 'amending Decision C(2006) 4332 fixing an annual indicative allocation by Member State for the period from 1 January 2007 to 31 December 2013 of the Community commitment appropriations from the European Fisheries Fund',
 '(notified under document number C(2008) 4358)',
 '(2008/693/EC)',
 'THE COMMISSION OF THE EUROPEAN COMMUNITIES,',
 'Having regard to the Treaty establishing the European Community,',
 'Having regard to Council Regulation (EC) No 1198/2006 of 27 July 2006 on the European Fisheries Fund (1), and in particular Article 14 thereof,',
 'Whereas, , Commission Decision C(2006) 4332 of 4 October 2006 (2), fixed for the period from 1 January 2007 to 31 December 2013 an annual indicative allocation by Member State, of the Community commitment appropriations for the regions eligible for funding from the European Fisheries Fund (hereinafter EFF) under the non-

[1, 2, 3, 4, 1, 2, 3, 1, 2, 3]

20.998067617416382


In [5]:
transformed_lines

['28.8.2008',
 'EN',
 'Official Journal of the European Union',
 'L 229/5',
 'COMMISSION DECISION',
 'of 13 August 2008',
 'amending Decision C(2006) 4332 fixing an annual indicative allocation by Member State for the period from 1 January 2007 to 31 December 2013 of the Community commitment appropriations from the European Fisheries Fund',
 '(notified under document number C(2008) 4358)',
 '(2008/693/EC)',
 'THE COMMISSION OF THE EUROPEAN COMMUNITIES,',
 'Having regard to the Treaty establishing the European Community,',
 'Having regard to Council Regulation (EC) No 1198/2006 of 27 July 2006 on the European Fisheries Fund (1), and in particular Article 14 thereof,',
 'Whereas, , Commission Decision C(2006) 4332 of 4 October 2006 (2), fixed for the period from 1 January 2007 to 31 December 2013 an annual indicative allocation by Member State, of the Community commitment appropriations for the regions eligible for funding from the European Fisheries Fund (hereinafter EFF) under the non-

In [ ]:

for i,line in enumerate( lines):
    if type( line )==list:
        if len( line ) > 20:
            lines[i]=lines[i][:50]

In [11]:
test_list=[ 'a', [1,2 , [3,5] ]  ]  #check if flattened list is longer than 30...  check what happens if you send list with length 30
list( flatten( test_list ) )

['a', 1, 2, 3, 5]

In [22]:
offsets[40]

(36054, 57224)

In [29]:
lines[40][0][-1]

')'

In [ ]:
process_nested_lines(  )

In [21]:
lines[40]

['Annual indicative breakdown by Member State for the period from 1 January 2007 to 31 December 2013 of the total commitment appropriations from the EFF (current prices)',
 '(in EUR)',
 'Member State',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 'Total',
 'Bulgaria',
 '5 818 765',
 '8 517 922',
 '11 598 136',
 '12 221 422',
 '13 084 212',
 '13 951 819',
 '14 817 432',
 '80 009 708',
 'Belgïe/Belgique',
 '0',
 '4 191 906',
 '4 263 970',
 '4 337 474',
 '4 412 449',
 '4 488 923',
 '4 566 926',
 '26 261 648',
 'Česká republika',
 '3 360 014',
 '3 526 490',
 '3 693 682',
 '3 868 715',
 '4 043 811',
 '4 218 249',
 '4 395 714',
 '27 106 675',
 'Danmark',
 '17 980 908',
 '18 340 527',
 '18 707 336',
 '19 081 484',
 '19 463 114',
 '19 852 376',
 '20 249 424',
 '133 675 169',
 'Deutschland',
 '21 738 775',
 '21 917 485',
 '22 094 648',
 '22 270 129',
 '22 443 794',
 '22 615 496',
 '22 785 090',
 '155 865 417',
 'Eesti',
 '9 130 309',
 '9 971 872',
 '10 889 823',
 '11 891 071'

In [6]:
transformed_lines, transformed_lines_offsets=transform_lines( lines, offsets )

In [ ]:
value_between_tagtype_generator=self.cas.get_view( OldSofaID ).select( value_between_tagtype )        

seek_vbtt=SeekableIterator( iter(value_between_tagtype_generator) )

init
test
get other lines:
get other lines
get other lines
get other lines
get other lines
get other lines
flatten offsets
transform lines


In [3]:

reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )

Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/testing_bug.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template


In [5]:
from xml.dom.minidom import parseString

parseString( '<p></p>'  )

In [ ]:
Activities seek to obtain and maintain the trust and confidence of EU citizens in this area . In the same way , legislation in the area of cross - border healthcare will be supported by targeted action 

In [9]:
def test_something():
    print( 'hello' )
    return

In [10]:
t=test_something()

hello


In [11]:
if t:
    print( "some" )